In [1]:
import numpy as np 
import pandas as pd 
import tensorflow_decision_forests as tfdf


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# --- 1. 訓練データの読み込みと前処理 ---
try:
    dataset_df = pd.read_csv('/kaggle/input/spaceship-titanic-ut-komaba-2025/train.csv')

    dataset_df = dataset_df.drop(['PassengerId', 'Name'], axis=1)


    dataset_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = dataset_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)

    label = "Transported"
    dataset_df[label] = dataset_df[label].astype(int)
    dataset_df['VIP'] = dataset_df['VIP'].astype(int)
    dataset_df['CryoSleep'] = dataset_df['CryoSleep'].astype(int)

    # 特徴量エンジニアリング: 'Cabin'列を'Deck', 'Cabin_num', 'Side'の3つに分割する
    dataset_df[["Deck", "Cabin_num", "Side"]] = dataset_df["Cabin"].str.split("/", expand=True)
    dataset_df = dataset_df.drop('Cabin', axis=1)


except FileNotFoundError:
    print("訓練ファイルが見つかりません。ファイルパスを確認してください。")
    exit()


# --- 2. データを訓練用と検証用に分割 ---
def split_dataset(dataset, test_ratio=0.20):
    """Pandasデータフレームを訓練セットとテストセットに分割します。"""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(dataset_df)
print(f"訓練データ: {len(train_ds_pd)}件, 検証データ: {len(valid_ds_pd)}件")

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label)


# --- 3. Gradient Boosted Treesモデルの定義と訓練 ---
print("\nGradient Boosted Treesモデル（LightGBM相当）を訓練しています...")

gbtm = tfdf.keras.GradientBoostedTreesModel(verbose=0)

gbtm.compile(metrics=["accuracy"])

gbtm.fit(train_ds)
print("モデルの訓練が完了しました。")


# --- 4. モデルの評価 ---
print("\nモデルの性能を評価しています...")
evaluation = gbtm.evaluate(valid_ds, return_dict=True)
print(f"検証データに対する正解率 (Accuracy): {evaluation['accuracy']:.4f}")


# --- 5. 提出用テストデータの読み込みと前処理 ---
print("\n提出用のテストデータを読み込み、前処理を行っています...")
try:
    test_df = pd.read_csv('/kaggle/input/spaceship-titanic-ut-komaba-2025/test.csv')
    submission_id = test_df.PassengerId

    test_df[['VIP', 'CryoSleep']] = test_df[['VIP', 'CryoSleep']].fillna(value=0)
    test_df[["Deck", "Cabin_num", "Side"]] = test_df["Cabin"].str.split("/", expand=True)
    test_df = test_df.drop('Cabin', axis=1)
    test_df['VIP'] = test_df['VIP'].astype(int)
    test_df['CryoSleep'] = test_df['CryoSleep'].astype(int)

    test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df)
    print("テストデータの前処理が完了しました。")

except FileNotFoundError:
    print("テストファイルが見つかりません。ファイルパスを確認してください。")
    exit()


# --- 6. 予測の実行と提出ファイルの作成 ---
print("\n予測を生成し、提出ファイルを作成しています...")

predictions = gbtm.predict(test_ds)
n_predictions = (predictions > 0.5).astype(bool)

output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': n_predictions.squeeze()})


output.to_csv('/kaggle/working/submission.csv', index=False)

print("\n提出ファイル 'submission.csv' が正常に作成されました！")
print("提出ファイルの先頭5行:")
print(output.head())

2025-10-10 14:12:09.754348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760105530.046643      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760105530.120130      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


訓練データ: 6937件, 検証データ: 1756件

Gradient Boosted Treesモデル（LightGBM相当）を訓練しています...


2025-10-10 14:12:29.406431: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1760105553.710761      13 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1760105553.712019      13 kernel.cc:783] Collect training examples
I0000 00:00:1760105553.712068      13 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1760105553.712943      13 kernel.cc:401] Number of batches: 7
I0000 00:00:1760105553.712984      13 kernel.cc:402] Number of examples: 6937
I0000 00:00:1760105553.715748      13 data_spec_inference.cc:354] 1258 item(s)

モデルの訓練が完了しました。

モデルの性能を評価しています...
2/2 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 0.8024
検証データに対する正解率 (Accuracy): 0.8024

提出用のテストデータを読み込み、前処理を行っています...
テストデータの前処理が完了しました。

予測を生成し、提出ファイルを作成しています...


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


5/5 [==============================] - 0s 8ms/step

提出ファイル 'submission.csv' が正常に作成されました！
提出ファイルの先頭5行:
  PassengerId  Transported
0     0013_01         True
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01         True
